# Exploring inference methods to plan inference pipeline

In [1]:
import hopsworks
from datetime import datetime
import pandas as pd

/Users/sambook/miniconda3/envs/id2223/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [52]:
# Today's date ----- is this needed/helpful anywhere?
# today = pd.to_datetime('2023-12-13').date()
# today = datetime.now().strftime('%Y-%m-%d')
today = datetime.now().date()
print(type(today))
print(today)

<class 'datetime.date'>
2023-12-14


## Get stored news articles

In [49]:
project = hopsworks.login()
fs = project.get_feature_store()

news_fg = fs.get_feature_group(name="news_articles", version=1)
news_df = news_fg.read()
news_df.head()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/185495
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (1.47s) 


,article_id,title,link,description,content,pubdate,source_id,source_priority,country,category,language
0,614fe26d75302581ff4c727dc2b62069,AP Breakthrough Entertainer: Charles Melton th...,https://www.winnipegfreepress.com/arts-and-lif...,LOS ANGELES (AP) — Fame didn’t come overnight ...,LOS ANGELES (AP) — Fame didn’t come overnight ...,2023-12-13 14:13:27,winnipegfreepress,101538,[canada],[entertainment],english
1,c1a910e57b0588354924cb70c07ff63e,"Badly burned but free, Israa Jaabis on her rel...",https://www.aljazeera.com/features/2023/12/13/...,Israa Jaabis was accused of attempted murder w...,Israa Jaabis was accused of attempted murder w...,2023-12-13 14:13:33,aljazeera_us,543,[united states of america],[top],english
2,975eafb21824c494b7b5492c189c397b,Dick Van Dyke Through the Years: Photos of the...,https://hollywoodlife.com/pics/dick-van-dyke-p...,Dick Van Dyke is one of Hollywood's most celeb...,Dick Van Dyke is one of Hollywood’s most celeb...,2023-12-13 14:15:32,hollywoodlife,977,[united states of america],[world],english
3,1cf88a47fd65f8a9b8be421268af0f4a,Virginia has tentative deal to move Washington...,https://lethbridgenewsnow.com/2023/12/13/virgi...,"RICHMOND, Va. (AP) - Virginia Gov. Glenn Young...","RICHMOND, Va. (AP) — Virginia Gov. Glenn Young...",2023-12-13 14:07:18,lethbridgenewsnow,149548,[canada],[top],english
4,2170ae2a33b0d42be841601f63d95311,Sydney Sweeney admits to being a 'workaholic',https://www.perthnow.com.au/entertainment/cele...,Sydney Sweeney has revealed that she's current...,"Sydney Sweeney is a ""workaholic"". The 26-year-...",2023-12-13 14:00:10,perthnow,49457,[australia],[entertainment],english


In [50]:
print("Number of articles:", len(news_df))

97


## Only keep today's articles

In [60]:
news_df = news_df[pd.to_datetime(news_df['pubdate']).dt.date == today]
print("Number of articles:", len(news_df))

53


## Add sentiments for articles

In [9]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [30]:
def format_sentiment(sentiment):
    if sentiment['label'] == 'NEGATIVE':
        return sentiment['score'] * -1
    else:
        return sentiment['score']

def get_sentiment_value(news_object):
    return format_sentiment(sentiment_pipeline(news_object['title'])[0])

In [63]:
news_df['sentiment'] = news_df.apply(get_sentiment_value, axis=1)

In [64]:
news_df.head()

,article_id,title,link,description,content,pubdate,source_id,source_priority,country,category,language,sentiment
44,90d87ff7ce4d2d71e7f9c945b8e6de58,All Blacks star Ardie Savea takes top honour a...,https://www.nzherald.co.nz/sport/all-blacks-st...,Ardie Savea has been named the Kelvin R. Trema...,A month after claiming World Rugby’s biggest p...,2023-12-14 08:15:00,nzherald,7971,[new zealand],[sports],english,0.999462
45,ee3c812c214b1870b858747efe58989d,Hong Kong places arrest bounties on activists ...,https://torontosun.com/news/world/hong-kong-pl...,HONG KONG — Hong Kong police accused five acti...,HONG KONG — Hong Kong police accused five acti...,2023-12-14 14:44:35,torontosun,12198,[canada],[top],english,-0.907741
46,bd19a1a7f26da17f94928139f5e46a9f,OpTic Texas vs Seattle Surge Betting Tips and ...,https://www.esportsbets.com/news/optic-texas-v...,OpTic Texas has stiff competition in Seattle S...,OpTic Texas has stiff competition in Seattle S...,2023-12-14 14:53:00,esportsbets,53698,[united states of america],[top],english,-0.600554
47,6bb160787348f9acb2a4eac669e5a380,KCC now offering local climate action grants w...,https://kildare-nationalist.ie/2023/12/14/kcc-...,Kildare County Council is now inviting not-for...,Kildare County Council is now inviting not-for...,2023-12-14 14:39:02,kildare-nationalist,2606560,[ireland],[sports],english,0.901746
48,d4d4aa89e6c601c4c4e60c7774e1c6e9,"NFL odds, lines, picks, spreads, bets, predict...",https://www.cbssports.com/nfl/news/nfl-odds-li...,SportsLine's advanced model has locked in its ...,Dak Prescott and the Dallas Cowboys have won t...,2023-12-14 14:57:39,cbssports,72,[united states of america],[top],english,0.665539


## Calculate today's average sentiment

In [83]:
avg_sentiment = news_df['sentiment'].mean()
avg_sentiment

-0.16408252828526046

## Find today's most positive article

In [82]:
most_positive = news_df.loc[news_df['sentiment'].idxmax()]
most_positive

article_id                          6be831b2317bd0413ca1e1cf3387e2bd
title              Secure Code Warrior’s Agile Learning Platform ...
link               https://www.ibtimes.com.au/press-release/20231...
description        Netskope partners with Secure Code Warrior to ...
content            INTERNATIONAL BUSINESS TIMES au NEWSLETTER SIG...
pubdate                                          2023-12-14 14:48:49
source_id                                                    ibtimes
source_priority                                               458722
country                                                  [australia]
category                                                [technology]
language                                                     english
sentiment                                                   0.999579
Name: 64, dtype: object